In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize,sent_tokenize
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
dataset_1 = pd.read_csv("Dataset_1.csv",encoding='latin-1')
dataset_2 = pd.read_csv("Dataset_2.csv",encoding='latin-1')

In [3]:
dataset_1.head()

,Document Title,Year,Abstract,PDF Link,label
0,Diagnostic Maintenance: A Technique Using a Co...,1963,A possible technique and the attending softwar...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,no
1,A Very High Speed Electro-Optical-Mechanical P...,1968,Adanced-performance automated phototypesetting...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,no
2,Recognition of Handprinted Numerals by Two-Sta...,1970,An optical character recognition system for ha...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,no
3,Computer Diagnosis Using the Blocking Gate App...,1971,In previous papers [3]â[5] the authors consi...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,no
4,Simulation modeling for air quality control,1971,Simulation modeling will have a major role in ...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,no


In [4]:
dataset_2.head()

,Document Title,Abstract,Year,PDF Link,Authors,Source,abs,label
0,A Conceptual Model of ICT-Supported Unified Pr...,This is an ongoing research in international o...,2006,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,Anicet Yalaho,EDOCW '06: Proceedings of the 10th IEEE on Int...,yes,yes
1,A Quantitative Assessment of Requirements Engi...,<p>Requirements engineering research has been ...,2007,http://dl.acm.org/citation.cfm?id=1768914&CFID...,"Alan Davis , Ann Hickey , Oscar Dieste , Na...","Book Series Lecture Notes in Computer Science,...",yes,yes
2,A survey and taxonomy of approaches for mining...,<p>A comprehensive literature survey on approa...,2007,http://dl.acm.org/citation.cfm?id=1345057&CFID...,"Huzefa Kagdi, Michael L. Collard, Jonathan I. ...",Journal of Software Maintenance and Evolution:...,yes,yes
3,An analysis of data sets used to train and val...,OBJECTIVE - to build up a picture of the natur...,2005,http://dl.acm.org/citation.cfm?id=1083166&CFID...,"Carolyn Mair, Martin Shepperd, Magne J?rgensen",PROMISE '05: Proceedings of the 2005 workshop ...,yes,yes
4,Challenges in Collaborative Modeling: A Litera...,Modeling is a key activity in conceptual desig...,2014,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,"Michiel Renger, Gwendolyn L. Kolfschoten and...",Book Series Lecture Notes in Business Informat...,yes,yes


In [5]:
'''THis block of code pre processes the data'''# stemmer = SnowballStemmer("english") 
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE) #remove white space
    RE_TAGS = re.compile(r"<[^>]+>") #removes tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE) #keep only asscii character
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", str(text))
    text = re.sub(RE_ASCII, " ", str(text))
    text = re.sub(RE_SINGLECHAR, " ", str(text))
    text = re.sub(RE_WSPACE, " ", str(text))

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [lemmatizer.lemmatize(w) for w in words_tokens_lower if w not in stop_words]
        
#         words_filtered = [
#             stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
#         ]

    text_clean = " ".join(words_filtered)
    return text_clean


In [6]:
def drop_nan(df):
    
    df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True)
    
    return df
    

In [7]:
drop_nan(dataset_1)
drop_nan(dataset_2)

dataset_1['Document Title'] = dataset_1['Document Title'].apply(clean_text)
dataset_1['Abstract'] = dataset_1['Abstract'].apply(clean_text)

dataset_2['Document Title'] = dataset_2['Document Title'].apply(clean_text)
dataset_2['Abstract'] = dataset_2['Abstract'].apply(clean_text)

In [12]:
dataset_1

,Document Title,Year,Abstract,PDF Link,label
0,diagnostic maintenance technique using computer,1963,possible technique attending software need adv...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
1,high speed electro optical mechanical phototyp...,1968,adanced performance automated phototypesetting...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
2,recognition handprinted numeral two stage feat...,1970,optical character recognition system handprint...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
3,computer diagnosis using blocking gate approach,1971,previous paper author considered application g...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
4,simulation modeling air quality control,1971,simulation modeling major role air quality pro...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
...,...,...,...,...,...
5995,research relationship curvature radius deflect...,2011,china current specification design asphalt pav...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
5996,sampling dmr practical low overhead permanent ...,2011,technology scaling manufacture time field perm...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
5997,qos aware multipath routing protocol delay sen...,2011,paper proposes qos multipath routing protocol ...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0
5998,monitoring high performance data stream vertic...,2011,last several year monitoring high performance ...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0


In [13]:
dataset_2

,Document Title,Abstract,Year,PDF Link,Authors,Source,abs,label
0,conceptual model ict supported unified process...,ongoing research international outsourcing sof...,2006,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,Anicet Yalaho,EDOCW '06: Proceedings of the 10th IEEE on Int...,1,1
1,quantitative assessment requirement engineerin...,requirement engineering research conducted yea...,2007,http://dl.acm.org/citation.cfm?id=1768914&CFID...,"Alan Davis , Ann Hickey , Oscar Dieste , Na...","Book Series Lecture Notes in Computer Science,...",1,1
2,survey taxonomy approach mining software repos...,comprehensive literature survey approach minin...,2007,http://dl.acm.org/citation.cfm?id=1345057&CFID...,"Huzefa Kagdi, Michael L. Collard, Jonathan I. ...",Journal of Software Maintenance and Evolution:...,1,1
3,analysis data set used train validate cost pre...,objective build picture nature type data set u...,2005,http://dl.acm.org/citation.cfm?id=1083166&CFID...,"Carolyn Mair, Martin Shepperd, Magne J?rgensen",PROMISE '05: Proceedings of the 2005 workshop ...,1,1
4,challenge collaborative modeling literature re...,modeling key activity conceptual design system...,2014,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,"Michiel Renger, Gwendolyn L. Kolfschoten and...",Book Series Lecture Notes in Business Informat...,1,1
...,...,...,...,...,...,...,...,...
1699,wireless wearable overview stage growth theory...,mobile medical application capacity provide se...,2005,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,"By Pauline (Hui Ying) Ooi , Gordana Culjak , E...",Found in: International Conference on Mobile B...,0,0
1700,working together inside emailbox,correction method inertial navigation system p...,2007,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,"Michael J. Muller, Daniel M. Gruen",September 2005 \r\nECSCW'05: Proceedings of th...,0,0
1701,working alternative development life cycle mul...,variety life cycle model software system devel...,2005,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,"Oddur Benediktsson, Darren Dalcher, Helgi Thor...",Thirteenth International Conference on Informa...,0,0
1702,writing computer science taxonomy writing task...,computer science graduate lack written communi...,2006,http://dl.acm.org/citation.cfm?id=1127468&CFID...,"Robert F. Dugan, Jr, Virginia G. Polanski","Journal of Computing Sciences in Colleges, V...",0,0


In [11]:
dataset_2['label'] = dataset_2['label'].replace({'yes': 1, 'no': 0})
dataset_2['abs'] = dataset_2['abs'].replace({'yes': 1, 'no': 0})
dataset_1['label'] = dataset_1['label'].replace({'yes': 1, 'no': 0})


In [17]:
dataset_1.to_csv('clean_dataset1.csv', index=False)
dataset_2.to_csv('clean_dataset1.csv', index=False)

[1 0]
